In [6]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

In [2]:
df = pd.read_csv("./hybridODE_weekly_data.csv")

In [3]:
df.head()

,time,beta,Ca,Susceptible,Exposed,Infectious_asymptomatic,Infectious_pre-symptomatic,Infectious_mild,Infectious_severe,Hospitalized_recovered,Hospitalized_deceased,Recovered,Deceased,a
0,0,0.500000,0.425,69.993257,0.003366,0.001966,0.000408,0.000159,0.000012,0.000171,0.000170,0.000227,0.000011,0
1,1,0.500000,0.425,69.991322,0.003445,0.002266,0.000671,0.000653,0.000051,0.000151,0.000155,0.000248,0.000036,0
2,2,0.500000,0.425,69.988503,0.004170,0.002598,0.000831,0.001174,0.000091,0.000135,0.000154,0.000267,0.000059,1
3,3,0.480491,0.425,69.984750,0.005304,0.003071,0.001039,0.001708,0.000132,0.000123,0.000165,0.000285,0.000083,1
4,4,0.448007,0.425,69.979881,0.006821,0.003741,0.001324,0.002317,0.000179,0.000115,0.000186,0.000301,0.000110,1


In [4]:
df.columns

Index(['time', 'beta', 'Ca', 'Susceptible', 'Exposed',
       'Infectious_asymptomatic', 'Infectious_pre-symptomatic',
       'Infectious_mild', 'Infectious_severe', 'Hospitalized_recovered',
       'Hospitalized_deceased', 'Recovered', 'Deceased', 'a'],
      dtype='object')

In [5]:
# Define an agent class with demographic attribute
class Agent:
    def __init__(self, agent_id, status, demographic):
        self.agent_id = agent_id
        self.status = status  # E.g., Susceptible, Exposed, etc.
        self.demographic = demographic  # Race or demographic group

# Define total population
total_population = 100000  # Example total population

# Race distribution for Medium
demographic_distribution = [50, 20, 15, 10, 5]  # Proportions for races A, B, C, D, E
races = ['A', 'B', 'C', 'D', 'E']

# Calculate the number of agents for each race
demographic_counts = [int((prop / 100) * total_population) for prop in demographic_distribution]

# Function to assign demographics to agents
def assign_demographics(demographic_counts, races):
    agent_demographics = []
    for i, count in enumerate(demographic_counts):
        agent_demographics.extend([races[i]] * count)
    
    # Shuffle to randomize assignment
    random.shuffle(agent_demographics)
    return agent_demographics

# Assign demographics to the agents
agent_demographics = assign_demographics(demographic_counts, races)

# Initialize the agents with demographic information
agents = []
for agent_id in range(total_population):
    demographic = agent_demographics[agent_id]
    agents.append(Agent(agent_id, 'Susceptible', demographic))  # All agents start as 'Susceptible'

# Lists to track agents who enter Hospitalized_recovered and Hospitalized_deceased
hospitalized_recovered_unique_ids = []
hospitalized_deceased_unique_ids = []
hospitalized_recovered_ids = []
hospitalized_deceased_ids = []

# Function to apply rates at each time step
def update_agent_states(agents, df, time_step, total_population, hospitalized_recovered_ids, hospitalized_deceased_ids, hospitalized_recovered_unique_ids, hospitalized_deceased_unique_ids):
    time_step_data = df[df['time'] == time_step].iloc[0]
    
    # Calculate proportions based on rates at the current time step
    proportions = {
        'Susceptible': time_step_data['Susceptible'],
        'Exposed': time_step_data['Exposed'],
        'Infectious_asymptomatic': time_step_data['Infectious_asymptomatic'],
        'Infectious_pre-symptomatic': time_step_data['Infectious_pre-symptomatic'],
        'Infectious_mild': time_step_data['Infectious_mild'],
        'Infectious_severe': time_step_data['Infectious_severe'],
        'Hospitalized_recovered': time_step_data['Hospitalized_recovered'],
        'Hospitalized_deceased': time_step_data['Hospitalized_deceased'],
        'Recovered': time_step_data['Recovered'],
        'Deceased': time_step_data['Deceased']
    }

    # Convert proportions to absolute numbers based on total population
    new_status_counts = {status: int(proportion/100 * total_population) for status, proportion in proportions.items()}

    # Shuffle agent states to match the proportions
    temp_agents = []

    for status, count in new_status_counts.items():
        current_agents = [agent for agent in agents if agent.status == status]

        # print("current agent count = ", len(current_agents))
        # print("count from data = ", count)
        
        # Remove agents from the compartment if there are more than needed
        while len(current_agents) > count:
            agent_to_remove = current_agents.pop()
            temp_agents.append(agent_to_remove)
                
        # Update all remaining agents' statuses for this step
        for agent in current_agents:
            agent.status = status
    
    # print(temp_agents)

    for status, count in new_status_counts.items():
        current_agents = [agent for agent in agents if agent.status == status]

        # Add agents to the compartment if there are fewer than needed
        while len(current_agents) < count:
            if len(temp_agents) == 0:
                print("Agent count mismatch!")
                break
            agent = temp_agents.pop()
            agent.status = status
            current_agents.append(agent)
    
    # Track the agents moving into 'Hospitalized_recovered' or 'Hospitalized_deceased'
    temp_HR = []
    temp_HD = []
    for agent in agents:
        if agent.status == 'Hospitalized_recovered' and agent.agent_id not in hospitalized_recovered_unique_ids:
            hospitalized_recovered_unique_ids.append(agent.agent_id)
            temp_HR.append(agent.demographic)
        elif agent.status == 'Hospitalized_deceased' and agent.agent_id not in hospitalized_deceased_unique_ids:
            hospitalized_deceased_unique_ids.append(agent.agent_id)
            temp_HD.append(agent.demographic)
                
    hospitalized_recovered_ids.append(temp_HR)
    hospitalized_deceased_ids.append(temp_HD)


# Simulate over time steps based on the rates
# for time_step in df['time'].unique():
for time_step in tqdm(range(52)):
    update_agent_states(agents, df, time_step, total_population, hospitalized_recovered_ids, hospitalized_deceased_ids, hospitalized_recovered_unique_ids, hospitalized_deceased_unique_ids)
    # print(f'Updated agents for time step {time_step}.')

# Output the agent IDs that were hospitalized and recovered or deceased at each timestep
print("Agents who were hospitalized and recovered:")
print(*hospitalized_recovered_ids,sep='\n')
print("\nAgents who were hospitalized and deceased:")
print(*hospitalized_deceased_ids,sep='\n')

100%|██████████| 52/52 [00:07<00:00,  7.23it/s]

Agents who were hospitalized and recovered:
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['D']
[]
[]
['C']
[]
['E']
['A']
['A', 'A']
['B']
['E', 'A']
['A', 'A', 'A']
['B', 'B', 'D']
['D', 'A', 'A', 'A']
['A', 'B', 'B', 'C']
['B', 'A', 'A', 'A', 'B']
['A', 'B', 'D', 'A', 'A', 'D']
['A', 'A', 'E', 'D', 'C', 'C', 'B']
['B', 'A', 'D', 'B', 'B', 'A', 'A', 'D']
['A', 'C', 'A', 'A', 'B', 'D', 'B', 'B']
['B', 'B', 'D', 'A', 'A', 'D', 'B', 'A']
['E', 'D', 'A', 'B', 'A', 'B', 'A']
['A', 'E', 'A', 'D', 'A', 'B', 'A', 'B']
['A', 'B', 'A', 'C', 'B', 'A']
['A', 'B', 'A', 'A', 'A', 'A']
['B', 'B', 'A']
['C', 'B', 'B']
['A']
[]
[]
[]
[]
[]
[]

Agents who were hospitalized and deceased:
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['A']
[]
[]
['D']
['A']
[]
['A']
['D', 'A']
['A', 'C']
['A', 'D']
['D', 'A', 'A']
['A', 'A', 'D', 'A']
['B', 'A', 'A', 'C', 'C']
['C', 'B', 'B', 'A', 'A', 'E']
['C', 'A', 'A', 'C', 'C', 'B', 'B', 'A']
['A', 'C', 'A', 'B', 'A', 'C', 'C', 'A', 'B']
['E', 'B', 'A', 'A', 'A', 

In [9]:
# Testing
# Check if counts for the HR and HD makes

In [8]:
# TODO: incorporate sampling method
# The probability of an agent being observed in HR and HD data is bias according to their demographic

# (r,d) -> factor between 0 to 1 -> row represents region, col represents demographic
indicator_matrix = [ [0.8, 0.7, 0.55, 0.3, 0.1],
                     [0.8, 0.7, 0.55, 0.3, 0.1], 
                     [0.8, 0.7, 0.55, 0.3, 0.1], 
                     [0.8, 0.7, 0.55, 0.3, 0.1], 
                     [0.8, 0.7, 0.55, 0.3, 0.1]
                    ]

In [18]:
hospital_records = []
for i, hosp in enumerate(hospitalized_recovered_ids):
    hospital_records.append(hospitalized_recovered_ids[i] + hospitalized_deceased_ids[i])

def get_sampling_prob(x_rd, t):
    time_step_data = df[df['time'] == t].iloc[0]
    
    # Calculate proportions based on rates at the current time step
    proportions = {
        'Infectious_severe': time_step_data['Infectious_severe'],
        'Hospitalized_deceased': time_step_data['Hospitalized_deceased'],
    }

    y_t = (proportions['Hospitalized_deceased'] - min(df['Hospitalized_deceased']))/(max(df['Hospitalized_deceased']) - min(df['Hospitalized_deceased']))

    return 1 - (1 - x_rd) * y_t

final_df = pd.DataFrame(columns=["week", "demo", "region", "sampled"])
for weekno, week in enumerate(hospital_records):
    for case_demo in week:
        region = random.randint(0, 4)
        sampling_prob = get_sampling_prob(indicator_matrix[region][ord(case_demo) - ord('A')], weekno)
        final_df.loc[len(final_df)] = [weekno, case_demo, region, np.random.choice([0, 1], p=[1 - sampling_prob, sampling_prob])]

In [20]:
final_df = final_df.sort_values(by="week")
final_df

,week,demo,region,sampled
0,12,A,3,1
1,15,D,0,1
2,16,A,4,1
3,18,A,2,1
4,19,D,0,1
...,...,...,...,...
658,44,B,3,1
671,45,D,4,0
672,45,B,1,1
670,45,A,1,0
